# This notebook is created to try out different things before implementing the efficiency test on pymesh

In [3]:
import pymesh
import numpy as np

In [4]:
box_1 = pymesh.generate_box_mesh(box_min = np.array([0, 0, 0]), box_max = np.array([25, 25, 25]))

In [5]:
box_small_1 = pymesh.generate_box_mesh(box_min = np.array([0, 0, 0]), box_max = np.array([1, 1, 1]))

box_small_2 = pymesh.generate_box_mesh(box_min = np.array([0.5, 0.5, 0.5]), 
box_max = np.array([1.5, 1.5, 1.5]))

In [6]:
output_mesh = pymesh.boolean(box_small_1, box_small_2,operation='intersection')

## Now we try building a union of three boxes. 

In [7]:
box_sd_1 = pymesh.generate_box_mesh(box_min = np.array([0, 0, 0]), box_max = np.array([1, 1, 1]))

box_sd_2 = pymesh.generate_box_mesh(box_min = np.array([1.5, 0, 0]), 
box_max = np.array([2.5, 1, 1]))

box_sd_3 = pymesh.generate_box_mesh(box_min = np.array([3.5, 0, 0]), 
box_max = np.array([4.5, 1, 1]))

In [8]:
csg = pymesh.CSGTree({'union': [{"mesh": box_sd_1}, {"mesh": box_sd_2}, {"mesh": box_sd_3}]})

In [10]:
# Generate a grid of cubes and their union 
number = 5
distance = 4 
list_of_grain_dict = []
scale = 1

for element in range(number):
    for iterator in range(number ** 2):
        reminder = iterator % number
        if reminder == 0:
            position = iterator // number
        else:
            position = reminder 
        new_grain =  pymesh.generate_box_mesh(
            box_min = np.array([element * distance, position * distance, number * distance]), 
            box_max = np.array([element * distance + scale, position * distance + scale, number * distance + scale]))
        new_grain_dict = {"mesh": new_grain}
        list_of_grain_dict.append(new_grain_dict)





In [11]:
%%time
CSG_grid_generation = pymesh.CSGTree({'union': list_of_grain_dict})

CPU times: user 3.11 s, sys: 118 ms, total: 3.23 s
Wall time: 2.44 s


In [13]:
%%time
# generate the cube block that sits beneath the grain
cube_block = pymesh.generate_box_mesh(box_min = [0, 0, 0], box_max = [number * distance + 1, number * distance + scale + 1, number * distance + scale])

cube_block_dict = {"mesh": cube_block}

CPU times: user 771 µs, sys: 3 µs, total: 774 µs
Wall time: 779 µs


In [23]:
CSG_entire_tree = pymesh.CSGTree({'difference': [cube_block_dict, CSG_grid_generation]})

In [24]:
output_mesh = CSG_entire_tree.mesh

In [25]:
pymesh.save_mesh("pymesh_25grid_cube_difference.stl", output_mesh)

In [26]:
import vedo
from vedo import *
load_pymesh = vedo.load("pymesh_25grid_cube_difference.stl")


In [27]:
vp = Plotter(shape=(1, 1), interactive=0, axes=3)
vp.show(load_pymesh, at=0)

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[10.869594372251527…